# Project4 Artificial intelligence
### Fatemeh Haghighi  810195385

In [98]:
from sklearn import tree
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from scipy.stats import mode
import numpy as np
import operator
import random
import copy

### Reading file 

In [213]:
data = pd.read_csv("data.csv")

## Part 1 : Implement Decision tree with sklearn library 
In this part first we implement dicision tree with "tree.DecisionTreeClassifier()" function then, we separate current data in two parts, X_train and X_test. one for learning the tree (by fitting) and the other for testing it. after making the tree we fit the train data into tree and predict the lable of test data based on learned information.
after that we could caculate the accuracy of our model by knowing the exact lables of test data in comprasion of predicted lables.
### Split data into test part and train part 

In [214]:
X_train, X_test = train_test_split(data, test_size=0.2, random_state=12347, shuffle = True)

### Make dicision tree 

In [215]:
clf = tree.DecisionTreeClassifier()

### Split target and featur parts in train set and test set

In [216]:
train_label = X_train.target
train_features = X_train.drop(['target'], axis = 1)

test_label = X_test.target
test_features = X_test.drop(['target'], axis = 1)


### Fit train features in dicision tree

In [218]:
clf = clf.fit(train_features, train_label)

### Accuracy of learned tree based on train set

In [221]:
train_prediction = clf.predict(train_features)
train_accuracy = accuracy_score(train_label, train_prediction)
print("accuracy in train data is ", train_accuracy)

accuracy in train data is  1.0


### Accuracy of learned tree based on test set

In [222]:
test_prediction = clf.predict(test_features)
test_accuracy = accuracy_score(test_label, test_prediction)
print("accuracy in test data is ", test_accuracy)

accuracy in test data is  0.7868852459016393


### Get mean of test data accuracy

In [224]:
result = []
for counter in range(1000):
    clf = clf.fit(train_features, train_label)
    test_prediction = clf.predict(test_features)
    result.append(accuracy_score(test_label, test_prediction))
result = np.array(result)
mean = result.mean()
std = result.std()
print(mean)
print(std)
    


0.7591475409836065
0.024857216552285635


## Part 2 : Implement random forest
In this part, first we partition our test data in 5 groups that each has 150 elements.
then we implement bagging class that has some methods.every object from this class patitions data into some group(based on input from constructor) and made tree, that every group sample use for learn a tree, it means that we get number of trees from input and partition data based on that too.then we predict test data label on each tree and get mode of the result. in the other words, for label of each row we accept the value that has the most frequent  among all 5 group for that row . after that we can calculate accuracy with axact labels and predicted labels.as you know most significant bagging's methods are : fit and predict as their functionality said briefly.
after that we calculate accuracy of our model by fiting train data with less features, it means for each feature, we calculate accuracy of model based on data that does not contain that feature. as a result we could find a feature that by ignoring that, we found accuracy that has minimum difference with original one on dicision tree. 
then we made a dicision tree that fites with train data containing 5 random feature that implemented below.
after all we implement random forest class, that each object of this class make numbers of tree(based on tree input number) that train data for each class, has random number of features(more than zero and less than maximum number of columns) and fixed number of row that selected randomly from original train data based on element number from input.


### Part 2.1 : Partition data into 5 group

In [225]:
def get_group_data(group_number, element_number):
    group_list = []
    for counter in range(group_number):
        sample_data = data.sample(n = element_number, replace = True) 
        group_list.append(sample_data)
    return group_list

In [226]:
group_list = get_group_data(5, 150)

### Part 2.2: Implement bagging model

In [36]:
class Bagging:
    def __init__(self, tree_number, element_each_tree):
        self.train_data = None
        self.tree_number = tree_number
        self.element_each_tree = element_each_tree
        self.train_data_groups = []
        self.train_featurs_groups = []
        self.train_lable_groups = []
        self.dicision_trees = []
        self.create_dicision_trees()

        
    def get_group_data(self):

        for counter in range(self.tree_number):
            sample_data = self.train_data.sample(n = self.element_each_tree, replace = True)
            self.train_data_groups.append(sample_data)
    
    def separate_labels(self):
        for df in self.train_data_groups:
            self.train_lable_groups.append(df.target)
            self.train_featurs_groups.append(df.drop(['target'], axis = 1))
                
    def create_dicision_trees(self):
        for counter in range(self.tree_number):
            clf = tree.DecisionTreeClassifier()
            self.dicision_trees.append(clf)
            
    def fit(self, train_df):
        self.train_data = train_df

        self.get_group_data()
        self.separate_labels()
        
        for i in range(self.tree_number):
            self.dicision_trees[i].fit(self.train_featurs_groups[i], self.train_lable_groups[i])
    
    def predict(self, test_data):
        test_data_label = test_data.target
        test_data_features = test_data.drop(['target'], axis = 1)

        predictions = []
        for tree in self.dicision_trees:
            predictions.append(tree.predict(test_data_features))
        result = mode(predictions, axis = 0)[0][0]
        return result
            
        

In [228]:
B = Bagging(5, 150)
B.fit(X_train)
result = B.predict(X_test)
bagging_train_accuracy = accuracy_score(test_label, result)
print("Accuracy of train data on learned tree is : ", bagging_train_accuracy)


Accuracy of train data on learned tree is :  0.8032786885245902


In [230]:
B.fit(X_train)
test_predict_result = B.predict(X_test)
bagging_test_accuracy = accuracy_score(test_label, test_predict_result)
print("Accuracy of test data on learned tree is : ", bagging_test_accuracy)

Accuracy of test data on learned tree is :  0.819672131147541


### Get mean of bagging accuracy over test data

In [232]:
r = []
for counter in range(100):
    B.fit(X_train)
    test_predict_result = B.predict(X_test)
    r.append(accuracy_score(test_label, test_predict_result))
r = np.array(r)
mean = r.mean()
std = r.std()
print(mean)
print(std)

0.7947540983606558
0.020798362658562254


### Part 2.3 : Remove each feature and calculate accuracy of new datafarame

In [166]:
accs = {}
def remove_feature(train_label, train_features, test_label, test_features):
    tree_rmv_features = tree.DecisionTreeClassifier()
    for col in train_features.columns:
        res = []
        for i in range(500):
            train_sample = train_features.drop([col], axis = 1)
            test_sample = test_features.drop([col], axis = 1)
            tree_rmv_features.fit(train_sample, train_label)
            p = tree_rmv_features.predict(test_sample)
            res.append(accuracy_score(test_label, p))
        res = np.array(res)
        accs[col] = res.mean()
    return accs


In [167]:
accs = remove_feature(train_label, train_features, test_label, test_features)
accs

{'age': 0.822032786885246,
 'ca': 0.7731147540983605,
 'chol': 0.7290819672131147,
 'cp': 0.8325573770491804,
 'exang': 0.7453770491803278,
 'fbs': 0.8092459016393443,
 'oldpeak': 0.7315409836065574,
 'restecg': 0.7531475409836064,
 'sex': 0.7562950819672131,
 'slope': 0.7582295081967212,
 'thal': 0.7407540983606556,
 'thalach': 0.786655737704918,
 'trestbps': 0.78127868852459}

In [235]:
print("deletion of feature : ", max(accs.items(), key=operator.itemgetter(1))[0], \
      "cause accuracy that has minimum diference with original one ")

deletion of feature :  cp cause accuracy that has minimum diference with original one 


### Part 2.4 : Make dicision tree based on 5 random features

In [236]:
res1 = []
for i in range(3000):
    test_f = copy.deepcopy(test_features)
    train_f = copy.deepcopy(train_features)
    
    columns = np.array(train_f.columns)
    selected = np.random.choice(columns , size = 5 , replace = False)
    
    train_f_sel = train_f[selected]
    test_f_sel = test_f[selected]
            
    small_tree = tree.DecisionTreeClassifier()
    small_tree.fit(train_f_sel, train_label)
    p = small_tree.predict(test_f_sel)
    res1.append(accuracy_score(test_label, p))
res1 = np.array(res1)
print(res1.mean())
print(res1.std())



0.7147540983606557
0.07184368950423008


### Part 2.5 : Build random forest

In [237]:
class Random_forest():
    def __init__(self, tree_number, element_number):
        self.tree_number = tree_number
        self.element_number = element_number
        self.trees_columns = []
        self.trees = []
        
    def fit(self, train_data):
        for i in range(self.tree_number):
            self.trees.append(tree.DecisionTreeClassifier())            
#           select some rows for train data of each tree        
            train_sampled = train_data.sample(n = self.element_number, replace = True)
#           separate target and features part of original data
            train_label = train_sampled.target
            train_features = train_sampled.drop(['target'], axis = 1)
#           select columns from features set
            columns = np.array(train_features.columns)
            features_number = np.random.randint(1 , len(train_sampled.columns) ) 
            self.trees_columns.append(np.random.choice(columns , size = features_number , replace = False))
#       fit the trees
        for i in range(self.tree_number):
            train_f_sel = train_sampled[self.trees_columns[i]] 
            self.trees[i].fit(train_f_sel, train_label)
            
    def predict(self, test_data):
        test_data_label = test_data.target
        test_data_features = test_data.drop(['target'], axis = 1)
        predictions = []
        for i in range(self.tree_number):
            predictions.append(self.trees[i].predict(test_data_features[self.trees_columns[i]]))
        result = mode(predictions, axis = 0)[0][0]
        return result

### Test random forest model

In [241]:
f = Random_forest(40, 150) 
f.fit(X_train)
pred = f.predict(X_test)
print(accuracy_score(pred , test_label))

0.819672131147541


### Get mean of random forest accuracy based  on test data

In [240]:
r = []
for counter in range(100):
    f.fit(X_train)
    test_predict_result = f.predict(X_test)
    r.append(accuracy_score(test_label, test_predict_result))
r = np.array(r)
mean = r.mean()
std = r.std()
print(mean)
print(std)

0.8327868852459019
0.039684322743967874


## Questions

### first question:
Bootstrap aggregating, also called bagging , is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

In the other words, When training, each tree in a random forest learns from a random sample of the data points. The samples are drawn with replacement, known as bootstrapping, which means that some samples will be used multiple times in a single tree.
bootstraping increase standard deviation and variance of accuracy of a model because of randomness forced to the model.

### Second question:
-Over-fitting is the phenomenon in which the learning system tightly fits the given training data so much that it would be inaccurate in predicting the outcomes of the untrained data. In decision trees, over-fitting occurs when the tree is designed so as to perfectly fit all samples in the training data set.
-dicision tree is sensitive to over fitting because dicision tree have no idea about train data and learn data completely by fitting so its sensitive to overfitting.
-in bagging we select some rows from original data randomly, so we do not have over fitting on whole data with bagging but we have over fitting on part of data that selected randomly. but in bagging probablity of reaching to wrong result is less that correct result because we get sample from original data randomly in number of trees and accept most frequent answer for each row. 

### Third question:
Random forest, like its name implies, consists of a large number of individual decision trees that operate as an ensemble. Each individual tree in the random forest spits out a class prediction and the class with the most votes becomes our model’s prediction.
Bootstrap aggregating, also called bagging , is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

The fundamental difference is that in Random forests, only a subset of features are selected at random out of the total and the best split feature from the subset is used to split each node in a tree, unlike in bagging where all features are considered for splitting a node.
In a more widesense comparison Random Forest uses Bagging to overcome overfiting in decision tree.
on the other words, random forest uses bagging to solve over fiting problem.


### Fourth question:
accuracy of part 1: 0.7591 <br>
accuracy of part 2.2: 0.7947 <br>
accuracy of part 2.5: 0.8327 <br> 
accuracies are in average case. <br>
from these numbers we can understood that having some(more that one) tree or model helps us to achieve much better result in prediction, in comparition of having single model